In [1]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_text_splitters import CharacterTextSplitter

In [4]:
loader = TextLoader(r"c:\Users\monms\Desktop\txt_doc.txt")
documents = loader.load()

documents

[Document(page_content="Generating random paragraphs can be an excellent way for writers to get their creative flow going at the beginning of the day. The writer has no idea what topic the random paragraph will be about when it appears. This forces the writer to use creativity to complete one of three common writing challenges. The writer can use the paragraph as the first one of a short story and build upon it. A second option is to use the random paragraph somewhere in a short story they create. The third option is to have the random paragraph be the ending paragraph in a short story. No matter which of these challenges is undertaken, the writer is forced to use creativity to incorporate the paragraph into their writing.\nA random paragraph can also be an excellent way for a writer to tackle writers' block. Writing block can often happen due to being stuck with a current project that the writer is trying to complete. By inserting a completely random paragraph from which to begin, it 

In [13]:
text_splitter = CharacterTextSplitter(chunk_size=35, chunk_overlap=10)
docs = text_splitter.split_documents(documents)
docs

Created a chunk of size 1073, which is longer than the specified 35


[Document(page_content="Generating random paragraphs can be an excellent way for writers to get their creative flow going at the beginning of the day. The writer has no idea what topic the random paragraph will be about when it appears. This forces the writer to use creativity to complete one of three common writing challenges. The writer can use the paragraph as the first one of a short story and build upon it. A second option is to use the random paragraph somewhere in a short story they create. The third option is to have the random paragraph be the ending paragraph in a short story. No matter which of these challenges is undertaken, the writer is forced to use creativity to incorporate the paragraph into their writing.\nA random paragraph can also be an excellent way for a writer to tackle writers' block. Writing block can often happen due to being stuck with a current project that the writer is trying to complete. By inserting a completely random paragraph from which to begin, it 

In [14]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [16]:

# save to disk 
# it stores them in a collection named langchain by the way, and generates unique ids for each collection
db = Chroma.from_documents(docs, embedding_function, persist_directory="./chroma_db")



In [18]:
# load from disk
db3 = Chroma(persist_directory="./chroma_db", embedding_function=embedding_function)

query = 'Generating random paragraphs can be an excellent way for writers to get their creative flow going at the beginning of the day.'
query1 = 'Another productive way to use this tool to begin a daily writing routine'


docs = db3.similarity_search(query1)
print(docs[0].page_content)

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


Another productive way to use this tool to begin a daily writing routine. One way is to generate a random paragraph with the intention to try to rewrite it while still keeping the original meaning. The purpose here is to just get the writing started so that when the writer goes onto their day's writing projects, words are already flowing from their fingers.
Another writing challenge can be to take the individual sentences in the random paragraph and incorporate a single sentence from that into a new paragraph to create a short story. Unlike the random sentence generator, the sentences from the random paragraph will have some connection to one another so it will be a bit different. You also won't know exactly how many sentences will appear in the random paragraph.


In [ ]:

# load from disk 
db3 = Chroma(persist_directory="./chroma_db", embedding_function=embedding_function)
docs = db3.similarity_search(query)

### passing chroma client to langchain

In [19]:
import chromadb

collection = persistent_client.get_or_create_collection("collection_name")
client = chromadb.PersistentClient(path='./chroma_db')

NameError: name 'persistent_client' is not defined

In [ ]:
# I think I gotta do it myself and that's it man 

# create a persistnet db, client
# change the distance you want to 
# make the embedding function 
# add to your collection the embeddings, 
# query something

In [44]:
# db4 = Chroma(persist_directory="./data")

import chromadb

client = chromadb.PersistentClient(path='./chroma_db')
print(client.list_collections())


collection= client.get_collection('langchain')
collection.peek()

collection = client.get_or_create_collection(
        name="lang_chain",
        metadata={"hnsw:space": "cosine"} # l2 is the default
    )
    

[Collection(name=collection_name), Collection(name=langchain), Collection(name=lang_chain)]


In [ ]:
data_client = chromadb.PersistentClient(path='./chroma_db')

# sentence window retrieval 

In [9]:
def retrieve_chunks(text, window_size):
    sentences = text.split('.')  # Split text into sentences
    chunks = [sentences[i:i+window_size] for i in range(0, len(sentences), window_size)]  # Create chunks
    return chunks

text = "This is a sample text. It consists of multiple sentences. We want to retrieve chunks of sentences from this text."


query = 'Generating random paragraphs. can be an excellent way for writers. to get their creative flow going at the beginning of the day.'

window_size = 2
chunks = retrieve_chunks(
    text=query,
    window_size=window_size
    )

for chunk in chunks:
    print("Chunk:", ' '.join(chunk))
    # Process and analyze the chunk here

Chunk: Generating random paragraphs  can be an excellent way for writers
Chunk:  to get their creative flow going at the beginning of the day 


# testing retriever

In [12]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [13]:
child_splitter = RecursiveCharacterTextSplitter(chunk_size=120, chunk_overlap=20)
parent_splitter = RecursiveCharacterTextSplitter(Chunk_size=400, chunk_overlap=20)


vectorstore = Chroma(
    collection_name = "full_documents", embedding_function
)


retriever = ParentDocumentRetriever(
    vector_store=vectorstore, 
    docstore=store, 
    child_splitter= child_splitter, 
    parent_splitter=parent_splitter
)
